In [1]:
import numpy as np
import pandas as pd
!pip install -qq scanpy
import scanpy as sc

     |████████████████████████████████| 2.0 MB 27.8 MB/s 
     |████████████████████████████████| 91 kB 6.7 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [13]:
# https://drive.google.com/file/d/1gk45jQH8TKnxsRF-1OrsZF6SJRgDtu23/view?usp=sharing
# https://drive.google.com/file/d/1BztSnALNc0KzQ-0U1CpWEyQgeEZLiXAW/view?usp=sharing
#!gdown https://drive.google.com/file/d/1BztSnALNc0KzQ-0U1CpWEyQgeEZLiXAW/view?usp=sharing
!unzip "/content/drive/Shareddrives/dl genomics/output35_csvs.zip"

Archive:  /content/drive/Shareddrives/dl genomics/output35_csvs.zip
   creating: output35_csvs/
  inflating: output35_csvs/fresh_68k_labels_35.csv  
  inflating: output35_csvs/fresh_68k_input_35.csv  


In [16]:
input_file = open("output35_csvs/fresh_68k_input_35.csv")
labels_file = open("output35_csvs/fresh_68k_labels_35.csv")
X = np.loadtxt(input_file, delimiter=",")
y = np.loadtxt(labels_file, delimiter=",")

In [17]:
import tensorflow as tf
import tensorflow.keras.layers as kl
from functools import partial

# tf.executing_eagerly()
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [27]:
class MultVAE(tf.keras.Model):

    '''
    Simple MultVAE
    '''

    def __init__(self, latent_size, input_shape, encoder=None, decoder=None, beta=0.):
        super().__init__()
        self.latent_size = latent_size
        self.output_size = input_shape[-1]
        self.beta = beta
        self.epsilon = 1e-5
        
        self.encoder = encoder if encoder is not None else MultVAE.get_default_encoder()
        self.decoder = decoder if decoder is not None else MultVAE.get_default_decoder(self.output_size)

        self.mu_layer = kl.Dense(self.latent_size, name='mu_layer')
        self.lv_layer = kl.Dense(self.latent_size, name='lv_layer')


    def call(self, x):
        ## Encode x into a learned mean/logvar analogue 

        ex = self.encoder(x)
        mu = self.mu_layer(ex)
        lv = self.lv_layer(ex)

        ## Reparameterize step to get latent z variate
        eps = tf.random.normal(tf.shape(mu))
        zu = mu + lv * tf.exp(0.5 * eps)

        ## Return decoding of generated variate in shape of input
        dz = tf.reshape(self.decoder(zu), (-1,x.shape[-1]))
        return tf.math.softmax(dz), mu, lv

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            dz, mu, lv = self(x)
            loss = self.loss_fn(y, dz, mu, lv)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        # tf.print(loss)
        # print(loss.numpy())
        self.compiled_metrics.update_state(y, dz)
        return {m.name : m.result() for m in self.metrics}#.update({"loss":loss})

    # staticmethod
    def loss_fn(self, y_true, y_pred, mu, lv):
        '''
        y_true  : groud truth
        y_pred  : model prediction
        mu      : distribution mean
        lv      : distribution log-variance
        beta    : annealing parameter for KL-RC ratio
        '''
        pe_rc = tf.math.log(y_pred + self.epsilon) * y_true   ## Per-entry Multinomial LL
        pe_kl = 1. + lv - tf.square(mu) - tf.exp(lv)  ## Per-entry KL-Divergence
        rc = -tf.reduce_mean(tf.reduce_sum(pe_rc, axis=-1))
        kl = -tf.reduce_mean(tf.reduce_sum(pe_kl, axis=-1)) / 2.
        #tf.print(kl)
        #tf.print(rc)
        tf.print("  loss: ",rc + self.beta * kl)
        return rc + self.beta * kl

    @staticmethod
    def get_default_encoder():
        '''
        NOTE: Authors implement dropout (i.e. 50%, training-only) 
        and col-wise l2 normalization (preprocessing) at beginning of encoder.
        Consider adding regularization (author uses 0.01 per-layer L2)
        '''
        return tf.keras.Sequential(
            [kl.Dense(2**n, activation='LeakyReLU') for n in (8, 7, 5)], 
            name='encoder')


    @staticmethod
    def get_default_decoder(output_size):
        return tf.keras.Sequential(
            [kl.Dense(2**n, activation='LeakyReLU') for n in (5, 7, 8)] +
            [kl.Dense(output_size)], 
            name='decoder')
        
################################################################################

# class MultELBOLoss(tf.keras.losses.Loss):

#     def __init__(self, beta=0.5):
#         super().__init__()
#         self.beta = beta

#     def call(self, y_true, y_pred, mu, lv):
#         '''
#         y_true  : groud truth
#         y_pred  : model prediction
#         mu      : distribution mean
#         lv      : distribution log-variance
#         beta    : annealing parameter for KL-RC ratio
#         '''
#         pe_rc = tf.nn.log_softmax(y_pred) * y_true   ## Per-entry Multinomial LL
#         pe_kl = 1 + lv - tf.square(mu) - tf.exp(lv)  ## Per-entry KL-Divergence
#         rc = -tf.reduce_mean(tf.reduce_sum(pe_rc, axis=-1))
#         kl = -tf.reduce_mean(tf.reduce_sum(pe_kl, axis=-1)) / 2
#         return rc + self.beta * kl

################################################################################
################################################################################

## Summary

In [21]:
vae_model = MultVAE(latent_size=20, input_shape=X.shape)
vae_model.compile(
    optimizer   = tf.keras.optimizers.Adam(learning_rate=1e-3),
    # loss        = MultELBOLoss(beta=0.5),
    metrics     = [tf.keras.metrics.MSE]
)
vae_model.build(X.shape)
vae_model.summary()

Model: "mult_vae_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Sequential)        (58701, 32)               293280    
                                                                 
 decoder (Sequential)        (58701, 1000)             294920    
                                                                 
 mu_layer (Dense)            multiple                  660       
                                                                 
 lv_layer (Dense)            multiple                  660       
                                                                 
Total params: 589,520
Trainable params: 589,520
Non-trainable params: 0
_________________________________________________________________


## Run

In [ ]:
# class LossHistory(tf.keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         super().on_train_begin(logs=logs)
#         self.losses = []
#         # self.val_losses = []

#     def on_batch_end(self, batch, logs={}):
#         super().on_batch_end(batch, logs=logs)
#         self.losses.append(logs.get('loss'))
#         # self.val_losses.append(logs.get('val_loss'))

#     def on_epoch_end(self, epoch, logs={}):
#         super().on_epoch_end(epoch, logs=logs)
#         print(tf.reduce_mean(self.losses))
#         self.losses = []
#         # self.losses.append(logs.get('loss'))
#         # self.val_losses.append(logs.get('val_loss'))

In [28]:
# X = tf.constant(X)
# Y = tf.constant(Y)

split = int(.95 * len(X))

vae_model = MultVAE(latent_size=20, input_shape=y.shape, beta=0.5)
vae_model.compile(
    optimizer   = tf.keras.optimizers.Adam(learning_rate=1e-3),
    # loss        = MultELBOLoss(beta=0.5),
    metrics     = [tf.keras.metrics.MSE]
)
history = vae_model.fit(X[:split], y[:split], batch_size=500, epochs=10)
vae_model.evaluate(X[split:], y[split:])


Epoch 1/10
  loss:  937.321289
112/112 [==============================] - 11s 95ms/step - mean_squared_error: 0.4410
Epoch 2/10
  loss:  646.285706
112/112 [==============================] - 10s 93ms/step - mean_squared_error: 0.4403
Epoch 3/10
  loss:  658.422913
112/112 [==============================] - 10s 94ms/step - mean_squared_error: 0.4402
Epoch 4/10
  loss:  648.029053
112/112 [==============================] - 11s 95ms/step - mean_squared_error: 0.4402
Epoch 5/10
  loss:  635.695374
112/112 [==============================] - 11s 96ms/step - mean_squared_error: 0.4401
Epoch 6/10
  loss:  641.798584
112/112 [==============================] - 11s 99ms/step - mean_squared_error: 0.4401
Epoch 7/10
  loss:  644.02179
112/112 [==============================] - 11s 101ms/step - mean_squared_error: 0.4401
Epoch 8/10
  loss:  639.028198
 43/112 [==========>...................] - ETA: 7s - mean_squared_error: 0.4409

KeyboardInterrupt: ignored

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
